In [2]:
import pandas as pd # Import the pandas library and give it the alias 'pd'

movie = pd.read_csv('/content/movie.csv')
rating = pd.read_csv('/content/rating.csv')
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movie.drop(columns=['genres'], inplace=True)

In [4]:
user = [
            {'title':'Breakfast Club, The', 'rating':4},
            {'title':'Toy Story', 'rating':2.5},
            {'title':'Jumanji', 'rating':3},
            {'title':"Pulp Fiction", 'rating':4.5},
            {'title':'Akira', 'rating':5}
         ]
inputMovie = pd.DataFrame(user)
inputMovie
#Filtering out the movies by title
Id = movie[movie['title'].isin(inputMovie['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovie = pd.merge(Id, inputMovie)

In [5]:
#Filtering out users that have watched movies that the input has watched and storing it
users = rating[rating['movieId'].isin(inputMovie['movieId'].tolist())]

userSubsetGroup = users.groupby(['userId'])
#Sorting it so that users with movie ratings most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [8]:
import pandas as pd # Import the pandas library and give it the alias 'pd'

# ... (Your previous code)

# Assuming 'topUsersRating' should be a DataFrame, create it with the necessary columns before using it
topUsersRating = pd.DataFrame({'similarityIndex': [0.5, 0.7, 0.3],  # Example values, replace with actual data
                                'rating': [4, 3, 5]})  # Example values, replace with actual data

# Now you can proceed with the calculation
topUsersRating['weightedRating'] = topUsersRating['similarityIndex'] * topUsersRating['rating']

In [13]:
# Example: Mapping ratings to movieIds (replace with your actual logic)
# Let's assume you have a dictionary 'rating_to_movieId' that maps ratings to movieIds
rating_to_movieId = {4: 1, 3: 2, 5: 3}  # Replace with your actual mapping

# Apply the mapping to create the 'movieId' column
topUsersRating['movieId'] = topUsersRating['rating'].map(rating_to_movieId)

# Now you can group by 'movieId'
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,0.5,2.0
2,0.7,2.1
3,0.3,1.5


In [15]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

# Create the recommendation_df DataFrame
recommendation_df = pd.DataFrame() # Initialize an empty DataFrame

# Calculate the weighted average recommendation score
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']

In [17]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']

In [23]:
print(topUsersRating.columns)

Index(['similarityIndex', 'rating', 'weightedRating', 'movieId'], dtype='object')


In [24]:
# Applying a sum to the topUsers after grouping it up by movieId
tempTopUsersRating = topUsersRating.groupby('movieId').agg({
    'similarityIndex': 'sum',
    'weightedRating': 'sum'
})
tempTopUsersRating.columns = ['sum_similarityIndex', 'sum_weightedRating']

In [25]:
# Creating the recommendation score
recommendation_df = pd.DataFrame()
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating'] / tempTopUsersRating['sum_similarityIndex']

# Sorting to get the top recommendations
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)

In [27]:
print(tempTopUsersRating.head())
print(tempTopUsersRating.columns)

         sum_similarityIndex  sum_weightedRating
movieId                                         
1                        0.5                 2.0
2                        0.7                 2.1
3                        0.3                 1.5
Index(['sum_similarityIndex', 'sum_weightedRating'], dtype='object')


In [28]:
# Creating the recommendation score
recommendation_df = tempTopUsersRating.copy()
recommendation_df['weighted average recommendation score'] = recommendation_df['sum_weightedRating'] / recommendation_df['sum_similarityIndex']
recommendation_df = recommendation_df.reset_index()

# Sorting to get the top recommendations
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)

print(recommendation_df.head())
print(recommendation_df.columns)

   movieId  sum_similarityIndex  sum_weightedRating  \
2        3                  0.3                 1.5   
0        1                  0.5                 2.0   
1        2                  0.7                 2.1   

   weighted average recommendation score  
2                                    5.0  
0                                    4.0  
1                                    3.0  
Index(['movieId', 'sum_similarityIndex', 'sum_weightedRating',
       'weighted average recommendation score'],
      dtype='object')


In [29]:
# Creating the recommendation score
recommendation_df = tempTopUsersRating.copy()
recommendation_df['weighted average recommendation score'] = recommendation_df['sum_weightedRating'] / recommendation_df['sum_similarityIndex']
recommendation_df = recommendation_df.reset_index()

# Sorting to get the top recommendations
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)

print(recommendation_df.head())
print(recommendation_df.columns)

   movieId  sum_similarityIndex  sum_weightedRating  \
2        3                  0.3                 1.5   
0        1                  0.5                 2.0   
1        2                  0.7                 2.1   

   weighted average recommendation score  
2                                    5.0  
0                                    4.0  
1                                    3.0  
Index(['movieId', 'sum_similarityIndex', 'sum_weightedRating',
       'weighted average recommendation score'],
      dtype='object')


In [32]:
print(set(recommendation_df['movieId']).intersection(set(movie['movieId'])))

{1, 2, 3}


In [36]:
# Applies a sum to the topUsers after grouping it up by movieId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

         sum_similarityIndex  sum_weightedRating
movieId                                         
1                        0.5                 2.0
2                        0.7                 2.1
3                        0.3                 1.5


In [37]:
# Creating the recommendation DataFrame
recommendation_df = tempTopUsersRating.copy()
recommendation_df['weighted average recommendation score'] = recommendation_df['sum_weightedRating'] / recommendation_df['sum_similarityIndex']
recommendation_df = recommendation_df.reset_index()

print(recommendation_df.head())
print(recommendation_df.columns)

   movieId  sum_similarityIndex  sum_weightedRating  \
0        1                  0.5                 2.0   
1        2                  0.7                 2.1   
2        3                  0.3                 1.5   

   weighted average recommendation score  
0                                    4.0  
1                                    3.0  
2                                    5.0  
Index(['movieId', 'sum_similarityIndex', 'sum_weightedRating',
       'weighted average recommendation score'],
      dtype='object')


In [38]:
# Sorting to get the top recommendations
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)

In [40]:
# Step 1: Create recommendation_df
recommendation_df = tempTopUsersRating.reset_index()
print("recommendation_df columns after reset_index:", recommendation_df.columns)

# Step 2: Calculate the weighted average
recommendation_df['weighted_avg'] = recommendation_df['sum_weightedRating'] / recommendation_df['sum_similarityIndex']
print("recommendation_df columns after calculation:", recommendation_df.columns)

# Step 3: Rename the column if needed
recommendation_df = recommendation_df.rename(columns={'weighted_avg': 'weighted average recommendation score'})
print("Final recommendation_df columns:", recommendation_df.columns)

print("recommendation_df head:")
print(recommendation_df.head())

recommendation_df columns after reset_index: Index(['movieId', 'sum_similarityIndex', 'sum_weightedRating'], dtype='object')
recommendation_df columns after calculation: Index(['movieId', 'sum_similarityIndex', 'sum_weightedRating', 'weighted_avg'], dtype='object')
Final recommendation_df columns: Index(['movieId', 'sum_similarityIndex', 'sum_weightedRating',
       'weighted average recommendation score'],
      dtype='object')
recommendation_df head:
   movieId  sum_similarityIndex  sum_weightedRating  \
0        1                  0.5                 2.0   
1        2                  0.7                 2.1   
2        3                  0.3                 1.5   

   weighted average recommendation score  
0                                    4.0  
1                                    3.0  
2                                    5.0  


In [42]:
print(movie[['title', 'weighted average recommendation score']])

                     title  weighted average recommendation score
0           Jumanji (1995)                                    4.0
1  Grumpier Old Men (1995)                                    3.0
2         Toy Story (1995)                                    NaN
